# 0. Prelims

In [1]:
from bs4 import BeautifulSoup
import pypub

### 0.1 HTML cleanup methods

In [2]:
def html_cleanup(content, stype):
    "content =  html_string"
    # parse HTML file content
    soup = BeautifulSoup(content, 'html.parser')
    # cleanup
    soup.head.contents=[]
    soup.find('nav').extract()
    for script in soup.find_all('script'):
        script.extract()
    if stype == "chapter":
        for h1 in soup.find_all('h1')[:-1]:
            h1.extract()
    else:
        for h1 in soup.find_all('h1'):
            h1.extract()
    return soup.prettify()

# 1. Summary data

In [3]:
# get summary from FPP_index.html
with open('FPP_index.html', 'r') as fin:
    summ_content = fin.read()

summ_soup = BeautifulSoup(summ_content, 'html.parser')
summ = summ_soup.find_all('ul')[0]

In [4]:
# Create a list to store the summary related data to generate epub file
fpp_meta = []
for elem in summ.find_all('li'):
    fpp_meta.append( {"id": elem['data-level'],
            "title": elem.string,
            "ulr": elem.find('a')['href'],
            "file": elem.find('a')['href'].split('/')[-1],
            "type": "section" if elem['data-level'].find('.') != -1 else "chapter"
           })
fpp_meta[1:3]

[{'file': u'intro.html',
  'id': u'1',
  'title': u'1 Getting started',
  'type': 'chapter',
  'ulr': u'https://otexts.org/fpp2/intro.html'},
 {'file': u'what-can-be-forecast.html',
  'id': u'1.1',
  'title': u'1.1 What can be forecast?',
  'type': 'section',
  'ulr': u'https://otexts.org/fpp2/what-can-be-forecast.html'}]

In [6]:
base_dir = "fpp2/"

# Epub object creation
fpp = pypub.Epub('Forecasting: Principles and Practice')

for item in fpp_meta:
    # get HTML file content
    with open(base_dir + item['file'], 'r') as htmlin:
        content = htmlin.read()
    #print "PROCESSING '{}' ...".format(item['title'])
    print "PROCESSING " + item['title'] + " ..."
    # cleaning
    clean_content = html_cleanup(content, stype=item['type'])
    # add to epub
    chap = pypub.create_chapter_from_string(clean_content, title = item['title'], url=item['ulr'])
    fpp.add_chapter(chap)

# create Epub file
fpp.create_epub('final')

PROCESSING Preface ...
PROCESSING 1 Getting started ...
PROCESSING 1.1 What can be forecast? ...
PROCESSING 1.2 Forecasting, planning and goals ...
PROCESSING 1.3 Determining what to forecast ...
PROCESSING 1.4 Forecasting data and methods ...
PROCESSING 1.5 Some case studies ...
PROCESSING 1.6 The basic steps in a forecasting task ...
PROCESSING 1.7 The statistical forecasting perspective ...
PROCESSING 1.8 Exercises ...
PROCESSING 1.9 Further reading ...
PROCESSING 2 Time series graphics ...
PROCESSING 2.1 ts objects ...
PROCESSING 2.2 Time plots ...
PROCESSING 2.3 Time series patterns ...
PROCESSING 2.4 Seasonal plots ...
PROCESSING 2.5 Seasonal subseries plots ...
PROCESSING 2.6 Scatterplots ...
PROCESSING 2.7 Lag plots ...
PROCESSING 2.8 Autocorrelation ...
PROCESSING 2.9 White noise ...
PROCESSING 2.10 Exercises ...
PROCESSING 2.11 Further reading ...
PROCESSING 3 The forecaster’s toolbox ...
PROCESSING 3.1 Some simple forecasting methods ...
PROCESSING 3.2 Transformations and ad

'final/Forecasting Principles and Practice.epub'